<a href="https://colab.research.google.com/github/crea0414/1st_CVDL/blob/master/Day015_Cifar_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten, GlobalAvgPool2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.datasets import cifar10
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
    mean = np.mean(X_train,axis=(0, 1, 2, 3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train-mean)/(std+1e-7)
    X_test = (X_test-mean)/(std+1e-7) 
    return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, stratify=y_train, test_size=0.2)

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_valid=one_hot.fit_transform(y_valid).toarray()
y_test=one_hot.transform(y_test).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [4]:
classifier=Sequential()

#卷積組合
classifier.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization(axis=-1))

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))
#comninatio
classifier.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
#classifier.add(Flatten())
classifier.add(GlobalAvgPool2D())

#FC
#classifier.add(Dense(100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#optimizer:
adam = Adam(lr=0.01)
#callbacks
lr_reducer = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.5, patience=2, verbose=1)
stop = EarlyStopping(monitor='val_acc', patience=5, mode='max', verbose=1)

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = classifier.fit(x_train,y_train,batch_size=100,epochs=30, callbacks=[lr_reducer, stop],
                         validation_data=(x_valid, y_valid))















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`




Train on 40000 samples, validate on 10000 samples
Epoch 1/30
40000/40000 [==============================] - 6s 146us/step - loss: 1.5057 - acc: 0.4503 - val_loss: 1.4523 - val_acc: 0.5006
Epoch 2/30
40000/40000 [==============================] - 4s 108us/step - loss: 1.1212 - acc: 0.6037 - val_loss: 1.1861 - val_acc: 0.5914
Epoch 3/30
40000/40000 [==============================] - 4s 107us/step - loss: 0.9700 - acc: 0.6609 - val_loss: 1.0769 - val_acc: 0.6320
Epoch 4/30
40000/40000 [==============================] - 4s 108us/step - loss: 0.8691 - acc: 0.6955 - val_loss: 1.0854 - val_acc: 0.6409
Epoch 5/30
40000/40000 [==============================] - 4s 108us/step - loss: 0.8090 - acc: 0.7148 - val_loss: 1.0960 - val_acc: 0.6480
Epoch 6/30
40000/40000 [==============================] - 4s 108us/step - loss: 0.7565 - acc: 0.7367 - val_loss: 0.9606 - val_acc: 0.6851
Epoch 7/30
40000/40000 [==============================] - 4s 107us/step - loss: 0.7066 - acc: 0.7536 - val_loss: 0.9824 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
test_loss, test_acc = classifier.evaluate(x_test, y_test )
print(f'test_loss: {test_loss:.3f}\ntest_acc:  {test_acc:.3f}')

10000/10000 [==============================] - 1s 78us/step
test_loss: 1.120
test_acc:  0.733
